In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

from trion.analysis.demod import pshet_coeff, pshet

In [ ]:
date = '211109'
folder = 'pshet_pulsed_ac_01'

file_pattern = 'step_approach_*.npz'

file_names = glob('../../data/' + date + '/' + folder + '/' + file_pattern)


afm_file = '../../data/' + date + '/' + folder + '/afm_tracking.txt'
afm_channels = ['z', 'amp']
header = ['target'] + [c + '_pre' for c in afm_channels] + [c + '_post' for c in afm_channels] 
df = pd.read_csv(afm_file, sep=' ', skiprows=1, names=header)
afm_avg = pd.DataFrame(df['target'])
for c in afm_channels:
    afm_avg[c] = (df[c + '_pre'] + df[c + '_post']) / 2

In [ ]:
def demodulate(files, nsamples: int, tap: int, ref: int) -> pd.DataFrame:
    steps = []
    for f in files:
        ds = np.load(f)
        df = pd.DataFrame({k: ds[k] for k in ds.keys()})
        if not nsamples == 0:
            df = df.iloc[:nsamples]
        try:
            binned_ft_avg = pshet(df, tap, ref)
        except Exception as e:
            if 'empty bins' in str(e):
                return None
            else:
                raise
        amps = np.abs(pshet_coeff(binned_ft_avg))
        steps.append(np.squeeze(amps.to_numpy()))
    curve_df = pd.DataFrame(steps)
    
    return curve_df

def normalize(sig):
    sig_0 = sig - sig.min()
    norm = sig_0 / sig_0.max()
    return norm

def make_plot(df_daq: pd.DataFrame, df_afm: pd.DataFrame, harmonics):
    lolim = 0
    hilim = 50
    height = df_afm.loc[lolim: hilim, 'target']*1e3
    
    plt.plot(height, normalize(df_afm.loc[lolim: hilim, 'amp']), label='amp', marker='.', lw=1)
    
    for h in harmonics:
        plt.plot(height, normalize(df_daq.loc[lolim:hilim, h]), label='O'+str(h), marker='.', lw=1)
    
    plt.xlabel('z  / nm')
    plt.ylabel('normalized signal  / arb.u.')
    plt.title(folder)
    plt.legend()
    plt.savefig(date + '_approach_' + folder + '.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
nsamples = 0  # 0 for all available samples per pixel
tap_nbins = 64
ref_nbins = 64
harmonics = [
        1,
        2,
        3,
        4
    ]

daq_data = demodulate(file_names, nsamples, tap_nbins, ref_nbins)
make_plot(daq_data, afm_avg, harmonics)